#### NLTK Text Pre-processing:

In [3]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from _cpwords import compound_keywords

In [4]:
def preprocess_text(text):
    # Define a list of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add specific terms to be removed
    remove_terms = {'introduct', 'literature', 'review', 'figure', 'doi', 'fig', 'table', 'conclusion', 
                    'altimg', 'gif', 'png', 'discussion', 'acknowledgment', 'appendix','http', 'copyright'}
    stop_words.update(remove_terms)

    # Initialize lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # Convert to lowercase
    text = text.lower()
    for original, compound in compound_keywords.items():
        text = text.replace(original, compound)
    
    # Tokenize the text into sentences
    sentence_tokens = sent_tokenize(text)
    sentences = []

    # Tokenize each sentence into words and remove stopwords
    for sentence in sentence_tokens:
        words = word_tokenize(sentence)
        filtered_words = []
        for word in words:
            word = lemmatizer.lemmatize(word)
            #word = stemmer.stem(word)
            
            # Remove non-alphabetic characters except underscores and hyphens
            word = re.sub(r'[^\w\s\-]', '', word)
            
            #if word.lower() not in stop_words:
            if word.lower() not in stop_words and not word.isdigit() and 1 < len(word) <= 20:
                filtered_words.append(word)
            
        sentences.append(filtered_words)
    
    return sentences

def process_json_files(directory):
    all_sentences = []

    # Iterate through each JSON file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Extract the original text from the JSON structure
                text = data['full-text-retrieval-response']['originalText']
                # Process the text to tokenize and remove stopwords
                processed_sentences = preprocess_text(text)
                # Append the processed sentences to the overall list
                all_sentences.extend(processed_sentences)
    
    return all_sentences

#### Train and Save Word2Vec Model:

In [5]:
directory = 'papers_json'

model = Word2Vec(
    sentences=process_json_files(directory),  
    vector_size=300,           # Size of the embedding vectors
    window=20,                 # Context window size
    min_count=2,               # Minimum occurrence in vocabulary
    workers=4                  # Number of threads for model training
)

In [10]:
model_save_path = "word2vec.model"
model.save(model_save_path)

#### Model Examination:

In [7]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from _cpwords import compound_keywords

model = Word2Vec.load("word2vec.model")
print("Model loaded successfully.")

Model loaded successfully.


In [8]:
# Find similar words 
similar_words_rl = model.wv.most_similar('rl', topn=100)
similar_words_carbon = model.wv.most_similar('carbon', topn=20)

print("Words similar to 'rl':")
for word, similarity in similar_words_rl:
    print(f"{word}: {similarity}")

print("\nWords similar to 'carbon':")
for word, similarity in similar_words_carbon:
    print(f"{word}: {similarity}")

Words similar to 'rl':
drl: 0.8592274785041809
dqn: 0.7611899375915527
learns: 0.729836642742157
model-free: 0.724945604801178
ddpg: 0.7184646129608154
ddqn: 0.6960535049438477
maddpg: 0.6896633505821228
rl-based: 0.6895571947097778
policy-gradient: 0.6804182529449463
learned: 0.6797705292701721
single-agent: 0.6784573197364807
sac: 0.6734286546707153
policy: 0.6724432110786438
pre-training: 0.6674242615699768
ppo: 0.6663268208503723
irl: 0.664676308631897
drl-based: 0.6629266738891602
bdq: 0.6615206003189087
gail: 0.6553522348403931
mcts: 0.655299186706543
reinforce: 0.6524274945259094
dyna-pinn: 0.6515287160873413
rc-ddq: 0.6482321619987488
learn: 0.6480127573013306
non-stationarity: 0.6472879648208618
warm-starting: 0.646366536617279
agent: 0.6379923224449158
marl: 0.6373324394226074
fashion: 0.631766676902771
deep_q-learning: 0.6290871500968933
pre-trained: 0.6253461837768555
well-trained: 0.624663233757019
tl: 0.6225482821464539
observational: 0.615376889705658
ddq: 0.613817512989

In [9]:
# Find similarity
similarity_1 = model.wv.similarity('control', 'operation')
print(similarity_1)

0.40574217
